In [1]:
import os
import json
import pandas as pd
import scienceplots
from deside.utility import (check_dir, sorted_cell_types)
from deside.utility.read_file import ReadExp, ReadH5AD
from deside.simulation import (BulkGEPGenerator, get_gene_list_for_filtering,
                               filtering_by_gene_list_and_pca_plot)

2024-08-29 18:31:18.949167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 18:31:24.596656: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /public3/soft/anaconda/3.8.3/anaconda3/lib:/opt/slurm/slurm/lib:
2024-08-29 18:31:24.597866: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-08-29 18:31:53.754075: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerro

No science style, please install the package of "SciencePlots" first, see https://github.com/garrettj403/SciencePlots
No science style, please install the package of "SciencePlots" first, see https://github.com/garrettj403/SciencePlots
No science style, please install the package of "SciencePlots" first, see https://github.com/garrettj403/SciencePlots
No science style, please install the package of "SciencePlots" first, see https://github.com/garrettj403/SciencePlots


# Synthesizing Bulk Tumors

## Workflow for Synthesizing D0, D1, and D2 Datasets

1. Define parameters for synthesizing bulk tumors.

2. Generate D0, D1, or D2 dataset:

   - a. Bulk GEP Generation (with or without GEP-filtering)
       - Cell Proportion Generation
           - D0: Random sampling
           - D1 and D2: Segment sampling
       - GEP-filtering
           - D1: Undergoes TCGA-guided GEP-filtering; only eligible GEPs are retained
           - D0 and D2: No filtering applied

   - b. Gene-level Filtering
       - D1: Undergoes gene-level filtering; subset of genes removed, all GEPs rescaled to TPM
       - D0 and D2: No filtering applied

## Example: Synthesizing a D1-like Dataset

In this example, we'll demonstrate how to synthesize a D1-like dataset with the following characteristics:
- 8,000 samples (GEPs)
- Segment sampling
- Two filtering steps applied

### Notes:
- We'll use the `S1` dataset directly instead of the raw integrated scRNA-seq dataset `S0`.
- This example was run on macOS with a 6-Core Intel Core i5 processor and 32 GB of memory.
- For larger sample sizes, more memory is required. Datasets `D0`, `D1`, and `D2` were synthesized on a computing server for optimal performance.

### Key Differences Between D0, D1, and D2:

| Dataset | Sampling Method | GEP-filtering | Gene-level Filtering |
|---------|-----------------|----------------|----------------------|
| D0      | Random          | No             | No                   |
| D1      | Segment         | Yes            | Yes                  |
| D2      | Segment         | No             | No                   |

### Define parameters for synthesizing bulk tumors.

In [25]:
# the list of 12 single cell RNA-seq datasets
sc_dataset_ids = ['hnscc_cillo_01', 'pdac_pengj_02', 'hnscc_puram_03',
                  'pdac_steele_04', 'luad_kim_05', 'nsclc_guo_06', 
                  'pan_cancer_07', 'prad_cheng_08', 'prad_dong_09', 
                  'hcc_sun_10', 'gbm_neftel_11', 'gbm_abdelfattah_12',]

# the list of 19 cancer types in the TCGA dataset
cancer_types = ['ACC', 'BLCA', 'BRCA', 'GBM', 'HNSC', 'LGG', 'LIHC', 'LUAD', 'PAAD', 'PRAD',
                'CESC', 'COAD', 'KICH', 'KIRC', 'KIRP', 'LUSC', 'READ', 'THCA', 'UCEC']

cancer_types_for_filtering = cancer_types.copy()

# coefficient to correct the difference of total RNA abundance in different cell types
# There is no effect to the final results if all the coefficients are set to 1
alpha_total_rna_coefficient = {'B Cells': 1.0, 'CD4 T': 1.0, 'CD8 T': 1.0, 'DC': 1.0,
                               'Endothelial Cells': 1.0, 'Cancer Cells': 1.0, 'Fibroblasts': 1.0,
                               'Macrophages': 1.0, 'Mast Cells': 1.0, 'NK': 1.0, 'Neutrophils': 1.0,
                               'Double-neg-like T': 1.0, 'Monocytes': 1.0}

# cell types and the corresponding subtypes
cell_type2subtypes = {'B Cells': ['Non-plasma B cells', 'Plasma B cells'],
                      'CD4 T': ['CD4 T'], 'CD8 T': ['CD8 T (GZMK high)', 'CD8 T effector'],
                      'DC': ['DC'], 'Endothelial Cells': ['Endothelial Cells'],
                      'Cancer Cells': ['Cancer Cells'],
                      'Fibroblasts': ['CAFs', 'Myofibroblasts'], 'Macrophages': ['Macrophages'],
                      'Mast Cells': ['Mast Cells'], 'NK': ['NK'], 'Neutrophils': ['Neutrophils'],
                      'Double-neg-like T': ['Double-neg-like T'], 'Monocytes': ['Monocytes']}

# the list of cell types
all_cell_types = sorted([i for v in cell_type2subtypes.values() for i in v])
all_cell_types = [i for i in sorted_cell_types if i in all_cell_types]
all_cell_types

['Plasma B cells',
 'Non-plasma B cells',
 'CD4 T',
 'CD8 T effector',
 'CD8 T (GZMK high)',
 'Double-neg-like T',
 'Cancer Cells',
 'DC',
 'Endothelial Cells',
 'CAFs',
 'Myofibroblasts',
 'Macrophages',
 'Mast Cells',
 'NK',
 'Neutrophils',
 'Monocytes']

In [26]:
# parameters
# for gene-level filtering
gene_list_type = 'high_corr_gene_and_quantile_range'
gene_quantile_range = [0.005, 0.5, 0.995]  # gene-level filtering

# for GEP-level filtering
gep_filtering_quantile = (0.0, 0.95)  # GEP-level filtering, L1-norm threshold
filtering_in_pca_space = True
pca_n_components = 0.9
n_base = 100  # averaging 100 GEPs sampled from S0 to synthesize 1 bulk GEP, used by S1 generation

# optional, if set a prior cell proportion range for each cell type, the GEP-filtering step will be faster, default is (0, 1)
# It can be set as cell_prop_prior = {'B Cells': (0, 0.25), 'CD4 T': (0, 0.25), 'CD8 T': (0, 0.25),
#                    'DC': (0, 0.1), 'Mast Cells': (0, 0.1), 'NK': (0, 0.1), 'Neutrophils': (0, 0.1),
#                    'Endothelial Cells': (0, 0.5), 'Fibroblasts': (0, 0.5), 'Macrophages': (0, 0.5),
#                    'Cancer Cells': (0, 1)}
cell_prop_prior = None

dataset2parameters = {
    
    # # Parameters to genereate D0-like dataset. To synthesize D0-like dataset, uncomment this section and comment out the other parts.
    # 'Mixed_N10K_random': {
    #     'sc_dataset_ids': sc_dataset_ids,
    #     'cell_type2subtype': cell_type2subtypes,
    #     'n_samples': 8000,
    #     'sampling_method': 'random', # or `random` used by Scaden
    #     'filtering': False,
    # },   

    # # Parameters to genereate D2-like dataset. To synthesize D2-like dataset, uncomment this section and comment out the other parts.
    #  'Mixed_N10K_segment_without_filtering': {
    #     'sc_dataset_ids': sc_dataset_ids,
    #     'cell_type2subtype': cell_type2subtypes,
    #     'n_samples': 8000,
    #     'sampling_method': 'segment', # or `random` used by Scaden
    #     'filtering': False,
    # },  

    # Parameters to genereate D1-like dataset. 
    'Mixed_N10K_segment': {
        'sc_dataset_ids': sc_dataset_ids,
        'cell_type2subtype': cell_type2subtypes,
        'n_samples': 8000,
        'sampling_method': 'segment', # or `random` used by Scaden
        'filtering': True,
    }  
}


for ds, params in dataset2parameters.items():
    if ('filtering' in params) and ('filtering_ref_types' not in params):
        if params['filtering']:
            params['filtering_ref_types'] = cancer_types_for_filtering
        else:
            params['filtering_ref_types'] = []
print(json.dumps(dataset2parameters, indent=2))

{
  "Mixed_N10K_segment_without_filtering": {
    "sc_dataset_ids": [
      "hnscc_cillo_01",
      "pdac_pengj_02",
      "hnscc_puram_03",
      "pdac_steele_04",
      "luad_kim_05",
      "nsclc_guo_06",
      "pan_cancer_07",
      "prad_cheng_08",
      "prad_dong_09",
      "hcc_sun_10",
      "gbm_neftel_11",
      "gbm_abdelfattah_12"
    ],
    "cell_type2subtype": {
      "B Cells": [
        "Non-plasma B cells",
        "Plasma B cells"
      ],
      "CD4 T": [
        "CD4 T"
      ],
      "CD8 T": [
        "CD8 T (GZMK high)",
        "CD8 T effector"
      ],
      "DC": [
        "DC"
      ],
      "Endothelial Cells": [
        "Endothelial Cells"
      ],
      "Cancer Cells": [
        "Cancer Cells"
      ],
      "Fibroblasts": [
        "CAFs",
        "Myofibroblasts"
      ],
      "Macrophages": [
        "Macrophages"
      ],
      "Mast Cells": [
        "Mast Cells"
      ],
      "NK": [
        "NK"
      ],
      "Neutrophils": [
        "Neutrophil

### Input files
For this demonstration, we'll be using three key input files. These files are available from Figshare, Zenodo or Github repositories.

1. **The GEPs of TCGA samples**
- File name: `merged_tpm.csv`
- Description: Gene expression profiles (GEPs) of 19 cancer types from TCGA in TPM format, used for TCGA-guided GEP-level filtering
- Download options:
    - Figshare: https://doi.org/10.6084/m9.figshare.23047547.v2
    - Zenodo: https://zenodo.org/records/10610306   
      - Path: DeSide_workflow/datasets/TCGA/tpm/merged_tpm.csv
        
2. **Cancer Type Annotation**
- File name: `tcga_sample_id2cancer_type.csv`
- Description: Annotation file mapping sample IDs to their respective cancer types for the 19 cancer types in the GEP file.
- Download option:
    - Github (this repository): https://github.com/OnlyBelter/DeSide_mini_example
      - Path: datasets/TCGA/tpm/tcga_sample_id2cancer_type.csv
    - Zenodo: https://zenodo.org/records/10610306
      - Path: DeSide_workflow/datasets/TCGA/tpm/tcga_sample_id2cancer_type.csv

3. **Synthesized Single-Cell-Type GEPs (Dataset S1)**
- File name: `simu_bulk_exp_SCT_N10K_S1_16sct.h5ad`
- Description: Contains the synthesized single-cell-type GEPs (sctGEPs) for 16 cell types, denoted as `S1`.
- Download options:
  - Figshare: https://doi.org/10.6084/m9.figshare.23043560.v2
  - Zenodo: https://zenodo.org/records/10610306
    - Path: DeSide_workflow/datasets/generated_sc_dataset_12ds_n_base100_all_subtypes/tpm/simu_bulk_exp_SCT_N10K_S1_16sct.h5ad
   
4. **Genes with a high correlation to cell proportions for each cell type**
- File name: `gene_list_filtered_by_high_corr_gene.csv`
- Description: genes with high correlation between their expression values and the cell proportions for each cell type, derived from the bulk GEP generation process.
- Download options:
  - Github: https://github.com/OnlyBelter/DeSide_mini_example
    - Path: datasets/simulated_bulk_cell_dataset/D2/gene_list_filtered_by_high_corr_gene.csv
  - Zenodo: https://zenodo.org/records/10610306
    - Path: DeSide_workflow/datasets/simulated_bulk_cell_dataset_subtypes_all_range/segment_12ds_no_filtering_n_base100_median_gep/D2/gene_list_filtered_by_high_corr_gene.csv


Note: While both Figshare and Zenodo repositories contain these files, the Zenodo repository provides a more structured file organization within the DeSide_workflow.

In [27]:
# Using TCGA as reference GEPs to filter synthetic GEPs 
tcga_data_dir = r'./datasets/TCGA/tpm/'  # input

tcga_merged_tpm_file_path = os.path.join(tcga_data_dir, 'merged_tpm.csv')
tcga2cancer_type_file_path = os.path.join(tcga_data_dir, 'tcga_sample_id2cancer_type.csv')

In [29]:
# the file path of the dataset `S1`
sct_dataset_file_path = './datasets/simu_bulk_exp_SCT_N10K_S1_16sct.h5ad'

In [30]:
# naming the file of filtered bulk cell dataset
q_names = ['q_' + str(int(q * 1000)/10) for q in gene_quantile_range]
replace_by = f'_filtered_by_{gene_list_type}.h5ad'
high_corr_gene_list = []
if 'quantile_range' in gene_list_type:
    replace_by = f'_filtered_by_{gene_list_type}_{q_names[0]}_{q_names[2]}.h5ad'
replace_by

'_filtered_by_high_corr_gene_and_quantile_range_q_0.5_q_99.5.h5ad'

In [31]:
sampling_method2dir = {
    'random': os.path.join('results', 'E3', '{}_{}ds_n_base{}'),
    'segment': os.path.join('results', 'E3', '{}_{}ds_{}_n_base{}_median_gep'),
}

### Implement bulk GEP generation with GEP-filtering.
Synthesize D1 with GEP-filtering when the "filtering" parameter is set to "True", and "sampling_method" set to "segment". 
- `filtering`: Determines whether to perform the GEP-filtering step.
- `sampling_method`: Sets the method to generate the cell proportion matrix. Use "segment" for segment sampling or "random" for random sampling.
- This step will take approximately 1.5 hours for 8,000 samples.

In [32]:
n_sc_datasets = len(sc_dataset_ids)
dataset2path = {}
log_file_path = './results/E3/DeSide_running_log.txt'
for dataset_name, params in dataset2parameters.items():
    if 'SCT' in dataset_name:
        pass  # using `S1` directly, omit this step here
    else:
        sampling_method = params['sampling_method']
        # the folder of simulated bulk cells
        simu_bulk_exp_dir = sampling_method2dir[sampling_method]
        if sampling_method in ['segment', 'dirichlet']:
            if params['filtering']:
                _postfix_filtered_ds_naming = f'_{len(cancer_types_for_filtering)}cancer'
                if filtering_in_pca_space:
                    _postfix_filtered_ds_naming += f'_pca_{pca_n_components}'
                simu_bulk_exp_dir = simu_bulk_exp_dir.format(sampling_method, n_sc_datasets,
                                                             gep_filtering_quantile[1],
                                                             str(n_base) + _postfix_filtered_ds_naming)
            else:
                simu_bulk_exp_dir = simu_bulk_exp_dir.format(sampling_method, n_sc_datasets, 'no_filtering', n_base)
        else:  # 'random'
            simu_bulk_exp_dir = simu_bulk_exp_dir.format(sampling_method, n_sc_datasets, n_base)
        check_dir(simu_bulk_exp_dir)
        # Instantiate a generator.
        bulk_generator = BulkGEPGenerator(simu_bulk_dir=simu_bulk_exp_dir,
                                          merged_sc_dataset_file_path=None,
                                          cell_type2subtype=params['cell_type2subtype'],
                                          sc_dataset_ids=params['sc_dataset_ids'],
                                          bulk_dataset_name=dataset_name,
                                          sct_dataset_file_path=sct_dataset_file_path,
                                          check_basic_info=False,
                                          tcga2cancer_type_file_path=tcga2cancer_type_file_path,
                                          total_rna_coefficient=alpha_total_rna_coefficient,
                                          cell_type_col_name='cell_type',
                                          subtype_col_name='cell_type')
        # GEP-filtering will be performed during this generation process. D1 will undergo GEP-filtering, while D0 and D2 will not.
        generated_bulk_gep_fp = bulk_generator.generated_bulk_gep_fp
        dataset2path[dataset_name] = generated_bulk_gep_fp
        if not os.path.exists(generated_bulk_gep_fp):
            bulk_generator.generate_gep(n_samples=params['n_samples'],
                                        simu_method='mul',
                                        sampling_method=params['sampling_method'],
                                        reference_file=tcga_merged_tpm_file_path,
                                        ref_exp_type='TPM',
                                        filtering=params['filtering'],
                                        filtering_ref_types=params['filtering_ref_types'],
                                        gep_filtering_quantile=gep_filtering_quantile,
                                        n_threads=5,
                                        log_file_path=log_file_path,
                                        show_filtering_info=False,
                                        filtering_method='median_gep',
                                        cell_prop_prior=cell_prop_prior,
                                        filtering_in_pca_space=filtering_in_pca_space,
                                        norm_ord=1, pca_n_components=pca_n_components)

  0%|          | 0/8000 [00:00<?, ?it/s]2024-08-29 19:23:15.556354: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 19:23:15.556352: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 19:23:15.556356: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other op

   > The following total RNA coefficient will be used:  {'CAFs': 1.0, 'CD4 T': 1.0, 'CD8 T (GZMK high)': 1.0, 'CD8 T effector': 1.0, 'Cancer Cells': 1.0, 'DC': 1.0, 'Double-neg-like T': 1.0, 'Endothelial Cells': 1.0, 'Macrophages': 1.0, 'Mast Cells': 1.0, 'Monocytes': 1.0, 'Myofibroblasts': 1.0, 'NK': 1.0, 'Neutrophils': 1.0, 'Non-plasma B cells': 1.0, 'Plasma B cells': 1.0}


 12%|█▎        | 1000/8000 [01:58<13:49,  8.44it/s]2024-08-29 19:25:15.673088: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 19:25:15.712881: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 19:25:15.775029: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them 

   > The following total RNA coefficient will be used:  {'CAFs': 1.0, 'CD4 T': 1.0, 'CD8 T (GZMK high)': 1.0, 'CD8 T effector': 1.0, 'Cancer Cells': 1.0, 'DC': 1.0, 'Double-neg-like T': 1.0, 'Endothelial Cells': 1.0, 'Macrophages': 1.0, 'Mast Cells': 1.0, 'Monocytes': 1.0, 'Myofibroblasts': 1.0, 'NK': 1.0, 'Neutrophils': 1.0, 'Non-plasma B cells': 1.0, 'Plasma B cells': 1.0}


 25%|██▌       | 2000/8000 [03:44<10:31,  9.50it/s]2024-08-29 19:26:51.225441: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 19:26:51.323659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 19:26:51.371514: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /pub

...

   > The following total RNA coefficient will be used:  {'CAFs': 1.0, 'CD4 T': 1.0, 'CD8 T (GZMK high)': 1.0, 'CD8 T effector': 1.0, 'Cancer Cells': 1.0, 'DC': 1.0, 'Double-neg-like T': 1.0, 'Endothelial Cells': 1.0, 'Macrophages': 1.0, 'Mast Cells': 1.0, 'Monocytes': 1.0, 'Myofibroblasts': 1.0, 'NK': 1.0, 'Neutrophils': 1.0, 'Non-plasma B cells': 1.0, 'Plasma B cells': 1.0}


 88%|████████▊ | 7000/8000 [11:38<01:33, 10.64it/s]2024-08-29 19:34:35.797116: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 19:34:35.828095: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 19:34:35.867133: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them 

   > The following total RNA coefficient will be used:  {'CAFs': 1.0, 'CD4 T': 1.0, 'CD8 T (GZMK high)': 1.0, 'CD8 T effector': 1.0, 'Cancer Cells': 1.0, 'DC': 1.0, 'Double-neg-like T': 1.0, 'Endothelial Cells': 1.0, 'Macrophages': 1.0, 'Mast Cells': 1.0, 'Monocytes': 1.0, 'Myofibroblasts': 1.0, 'NK': 1.0, 'Neutrophils': 1.0, 'Non-plasma B cells': 1.0, 'Plasma B cells': 1.0}


100%|██████████| 8000/8000 [13:21<00:00,  9.98it/s]


   > Got 8000 samples from 8000 within l1 distance between 0 and 0 by quantile 0.0%-95.0%

---->>> <class 'deside.simulation.generate_data.BulkGEPGenerator'>:
{
  "simu_bulk_dir": "results/E3/segment_12ds_no_filtering_n_base100_median_gep",
  "cell_type_used": [
    "B Cells",
    "CD4 T",
    "CD8 T",
    "DC",
    "Endothelial Cells",
    "Cancer Cells",
    "Fibroblasts",
    "Macrophages",
    "Mast Cells",
    "NK",
    "Neutrophils",
    "Double-neg-like T",
    "Monocytes"
  ],
  "cell_subtype_used": [
    "Non-plasma B cells",
    "Plasma B cells",
    "CD8 T (GZMK high)",
    "CD8 T effector",
    "CAFs",
    "Myofibroblasts"
  ],
  "sc_dataset_used": [
    "hnscc_cillo_01",
    "pdac_pengj_02",
    "hnscc_puram_03",
    "pdac_steele_04",
    "luad_kim_05",
    "nsclc_guo_06",
    "pan_cancer_07",
    "prad_cheng_08",
    "prad_dong_09",
    "hcc_sun_10",
    "gbm_neftel_11",
    "gbm_abdelfattah_12"
  ],
  "bulk_dataset_name": "Mixed_N10K_segment_without_filtering",
  "n_samp

### Perform gene-level filtering (part1).
In this step, only the genes with high correlation between their expression values and the cell proportions for each cell type will be kept.
- This filtering process is applied to D1, while D0 and D2 do not undergo this step.
- The gene list used for this part can be generated once and then applied to all datasets. In this case, we use the gene list generated based on the dataset D2.

In [33]:
d2_dir = './datasets/simulated_bulk_cell_dataset/D2/'
high_corr_gene_file_path = os.path.join(d2_dir, f'gene_list_filtered_by_high_corr_gene.csv')
# if not os.path.exists(high_corr_gene_file_path):
#     print(f'High correlation gene list will be saved in: {high_corr_gene_file_path}')
#     high_corr_gene_list = get_gene_list_for_filtering(bulk_exp_file=generated_bulk_gep_fp,
#                                                       filtering_type='high_corr_gene',
#                                                       corr_result_fp=corr_result_fp,
#                                                       tcga_file=tcga_merged_tpm_file_path,
#                                                       quantile_range=gene_quantile_range,
#                                                       result_file_path=high_corr_gene_file_path,
#                                                       corr_threshold=0.3, n_gene_max=1000)
# else:
#     print(f'High correlation gene list file existed: {high_corr_gene_file_path}')
high_corr_gene_list = pd.read_csv(high_corr_gene_file_path)
high_corr_gene_list = high_corr_gene_list['gene_name'].to_list()
len(high_corr_gene_list)

9481

In [34]:
for i in dataset2parameters.keys():          
    dataset_name = i 
    params = dataset2parameters[dataset_name]

params

{'sc_dataset_ids': ['hnscc_cillo_01',
  'pdac_pengj_02',
  'hnscc_puram_03',
  'pdac_steele_04',
  'luad_kim_05',
  'nsclc_guo_06',
  'pan_cancer_07',
  'prad_cheng_08',
  'prad_dong_09',
  'hcc_sun_10',
  'gbm_neftel_11',
  'gbm_abdelfattah_12'],
 'cell_type2subtype': {'B Cells': ['Non-plasma B cells', 'Plasma B cells'],
  'CD4 T': ['CD4 T'],
  'CD8 T': ['CD8 T (GZMK high)', 'CD8 T effector'],
  'DC': ['DC'],
  'Endothelial Cells': ['Endothelial Cells'],
  'Cancer Cells': ['Cancer Cells'],
  'Fibroblasts': ['CAFs', 'Myofibroblasts'],
  'Macrophages': ['Macrophages'],
  'Mast Cells': ['Mast Cells'],
  'NK': ['NK'],
  'Neutrophils': ['Neutrophils'],
  'Double-neg-like T': ['Double-neg-like T'],
  'Monocytes': ['Monocytes']},
 'n_samples': 8000,
 'sampling_method': 'segment',
 'filtering': False,
 'filtering_ref_types': []}

### Perform gene-level filtering (part2).
In this step, only the genes whose expression values fall within the specified quantile range of their counterparts in TCGA samples will be kept.
- This filtering process is applied to D1, while D0 and D2 do not undergo this step.
- The quantile range is set by the parameter `gene_quantile_range`.
- Finally, the intersection of genes passing both filtering steps (part1 and part2) will be used in the gene-level filtering step.

In [35]:
# gene-level filtering that depends on the high correlation genes and quantile range (each dataset itself). D1 will undergo gene-filtering, while D0 and D2 will not.
if params['filtering'] and 'Mixed' in dataset_name:
    filtered_file_path = generated_bulk_gep_fp.replace('.h5ad', replace_by)
    if not os.path.exists(filtered_file_path):
        gene_list = high_corr_gene_list.copy()
        # get gene list, filtering, PCA and plot
        current_result_dir = os.path.join(simu_bulk_exp_dir, dataset_name)
        check_dir(current_result_dir)
        # the gene list file for current dataset
        if 'quantile_range' in gene_list_type:
            gene_list_file_path = os.path.join(simu_bulk_exp_dir, dataset_name, 
                                               'gene_list_filtered_by_quantile_range.csv')
            gene_list_file_path = gene_list_file_path.replace('.csv', f'_{q_names[0]}_{q_names[2]}.csv')
            if not os.path.exists(gene_list_file_path):
                print(f'Gene list of {dataset_name} will be saved in: {gene_list_file_path}')
                # obtain gene list within quantile range and other process files for gene-level filtering
                quantile_gene_list = get_gene_list_for_filtering(bulk_exp_file=generated_bulk_gep_fp,
                                                                 filtering_type='quantile_range',
                                                                 tcga_file=tcga_merged_tpm_file_path,
                                                                 quantile_range=gene_quantile_range,
                                                                 result_file_path=gene_list_file_path,
                                                                 q_col_name=q_names)
            else:
                print(f'Gene list file existed: {gene_list_file_path}')
                quantile_gene_list = pd.read_csv(gene_list_file_path)
                quantile_gene_list = quantile_gene_list['gene_name'].to_list()
            # get the intersection of the two gene lists (high correlation genes and within quantile range)
            gene_list = [gene for gene in gene_list if gene in quantile_gene_list]
        # save the filtered gene list
        gene_list_file_path = os.path.join(simu_bulk_exp_dir, dataset_name,
                                           f'gene_list_filtered_by_{gene_list_type}.csv')
        pd.DataFrame({'gene_name': gene_list}).to_csv(gene_list_file_path, index=False)
        
        bulk_exp_obj = ReadH5AD(generated_bulk_gep_fp)
        bulk_exp = bulk_exp_obj.get_df()
        bulk_exp_cell_frac = bulk_exp_obj.get_cell_fraction()
        tcga_exp = ReadExp(tcga_merged_tpm_file_path, exp_type='TPM').get_exp()
        pc_file_name = f'both_TCGA_and_simu_data_{dataset_name}'
        pca_model_file_path = os.path.join(current_result_dir, f'{pc_file_name}_PCA_{gene_list_type}.joblib')
        pca_data_file_path = os.path.join(current_result_dir, f'{dataset_name}_PCA_with_TCGA_{gene_list_type}.csv')
        # save GEPs data by filtered gene list
        print('Filtering by gene list and PCA plot')
        # execute the gene-level filtering process 
        filtering_by_gene_list_and_pca_plot(bulk_exp=bulk_exp, tcga_exp=tcga_exp, gene_list=gene_list,
                                            result_dir=current_result_dir, n_components=2,
                                            simu_dataset_name=dataset_name,
                                            pca_model_name_postfix=gene_list_type,
                                            pca_model_file_path=pca_model_file_path,
                                            pca_data_file_path=pca_data_file_path,
                                            h5ad_file_path=filtered_file_path,
                                            cell_frac_file=bulk_exp_cell_frac,
                                            figsize=(5, 5))

In [17]:
!tree ./results/E3/

./results/E3/
├── DeSide_running_log.txt
├── TCGA_project_to_D1_PCA.png
└── segment_12ds_0.95_n_base100_19cancer_pca_0.9_median_gep
    ├── D1_PCA.joblib
    ├── Mixed_N10K_segment
    │   ├── Mixed_N10K_segment_PCA_with_TCGA_high_corr_gene_and_quantile_range.csv
    │   ├── Mixed_N10K_segment_PCA_with_TCGA_high_corr_gene_and_quantile_range_PC0_PC1.png
    │   ├── gene_list_filtered_by_high_corr_gene_and_quantile_range.csv
    │   └── gene_list_filtered_by_quantile_range_q_0.5_q_99.5.csv
    ├── generated_frac_Mixed_N10K_segment.csv
    ├── simu_bulk_exp_Mixed_N10K_segment_log2cpm1p.csv
    ├── simu_bulk_exp_Mixed_N10K_segment_log2cpm1p.h5ad
    ├── simu_bulk_exp_Mixed_N10K_segment_log2cpm1p_filtered_by_high_corr_gene_and_quantile_range_q_0.5_q_99.5.h5ad
    └── simu_bulk_exp_Mixed_N10K_segment_sampled_sc_cell_id.csv

3 directories, 12 files


### Output files description
```text
results/E3  # the output folder of this example
|-- DeSide_running_log.txt  # log file
`-- segment_12ds_0.95_n_base100_19cancer_pca_0.9_median_gep
    |-- Mixed_N10K_segment
    |   |-- Mixed_N10K_segment_PCA_with_TCGA_high_corr_gene_and_quantile_range.csv  # Values of the first two principal components (PCs)
    |   |-- Mixed_N10K_segment_PCA_with_TCGA_high_corr_gene_and_quantile_range_PC0_PC1.png  # Visualization of PCA for the generated dataset and TCGA
    |   |-- gene_list_filtered_by_quantile_range_q_0.5_q_99.5.csv  # Gene list after filtering by quantile range
    |   `-- gene_list_filtered_by_high_corr_gene_and_quantile_range.csv  # Gene list after filtering by correlation and quantile range
    |-- generated_frac_Mixed_N10K_segment.csv  # Cell proportion matrix
    |-- simu_bulk_exp_Mixed_N10K_segment_log2cpm1p.csv  # Generated bulk gene expression profiles (GEPs) without filtering (csv format)
    |-- simu_bulk_exp_Mixed_N10K_segment_log2cpm1p.h5ad  # Generated bulk GEPs without filtering (h5ad format)
    |-- simu_bulk_exp_Mixed_N10K_segment_log2cpm1p_filtered_by_high_corr_gene_and_quantile_range_q_0.5_q_99.5.h5ad  # Generated bulk GEPs after filtering (h5ad format)
    `-- simu_bulk_exp_Mixed_N10K_segment_sampled_sc_cell_id.csv  # Selected single-cell GEPs from dataset S1 during GEP sampling
```